In [1]:
import pandas as pd
import glob
import os
import re

import json

In [2]:
#### MERGE ALL THE FILES IN ONE
# setting the path for joining multiple files
files = os.path.join("data-2/", "Kickstarter*.csv")

# list of merged files returned
files = glob.glob(files)

# joining files with concat and read_csv
df = pd.concat(map(pd.read_csv, files), ignore_index=True)


In [3]:
#save merged raw dataframe as a file 
df.to_csv('data/0_data_raw.csv')

### creator_id
extracting creator_id

In [4]:
#creating a new column
df['creator_id'] = df['creator']
#set a list from 0 til number of rows
rows = list(range(0,df.creator_id.shape[0]))

In [5]:
#patience please - the code needs some; around 4.5 min
# with a for loop splitting the dictionary in parts and extracting the needed info
for cell in rows:
    x = df['creator_id'][cell]
    x = re.split(":", df['creator_id'][cell])
    x = re.split(",", x[1])
    df['creator_id'][cell] = int(x[0])

/var/folders/39/h9t97r415hgdzn9cx8g2r7ch0000gn/T/ipykernel_5913/456914433.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['creator_id'][cell] = int(x[0])


### dict columns

In [6]:
#convert needed columns with dict content into columns
cols = ['category']
for col in cols:
    pd.json_normalize
    stdf = df[col].apply(json.loads)
    extracted = pd.DataFrame(stdf.tolist())
    extracted = extracted.add_prefix(f'{col}_')
    df = pd.concat([df, extracted], axis = 1)
    df = df.drop(col, axis = 1)

#drop non-convertable columns + columns which don't have any extra information 
# (photo, urls, category_urls)
col = ['creator', 'location', 'profile', 'photo', 'urls', 'category_urls']
df = df.drop(col, axis = 1)

### unixstamp

In [7]:
# unixstamp into datetype
dates = ['created_at', 'deadline', 'launched_at', 'state_changed_at']
for date in dates:
    df[date] = pd.to_datetime(df[date],unit='s')

In [8]:
#save working dataframe as file
df.to_csv('data/1_working_data.csv')